In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train = "/content/drive/My Drive/Kaggle/Speaker Recognition/train"
validation = "/content/drive/My Drive/Kaggle/Speaker Recognition/validation"

In [0]:
import keras
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        train,
        target_size=(224, 224),
        color_mode = 'rgb',
        batch_size=32,
        class_mode='categorical',
        shuffle = True)



Found 1625 images belonging to 10 classes.


In [0]:
valid_datagen = ImageDataGenerator(rescale=1./255)


valid_generator = valid_datagen.flow_from_directory(
    validation,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 375 images belonging to 10 classes.


In [0]:
vgg16_model = keras.applications.vgg16.VGG16()





553467904/553467096 [==============================] - 8s 0us/step








In [0]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Activation, Dropout, Flatten, Dense



In [0]:
model =  Sequential()
for layer in vgg16_model.layers[:-4]:
  model.add(layer)


In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [0]:
for layer in model.layers:
  layer.trainable = False

In [0]:
model.add(Conv2D(64, (2, 2)))
model.add(Conv2D(64, (1, 1)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

In [0]:
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

In [0]:
model.add(Flatten()) 

In [0]:
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(10))
model.add(Activation('softmax'))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [0]:
from keras import metrics
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=[metrics.categorical_accuracy])

In [0]:
model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=60,
        validation_data=valid_generator,
        validation_steps=70)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/60
100/100 [==============================] - 676s 7s/step - loss: 2.2342 - categorical_accuracy: 0.1521 - val_loss: 1.9028 - val_categorical_accuracy: 0.2328
Epoch 2/60
100/100 [==============================] - 67s 672ms/step - loss: 1.8473 - categorical_accuracy: 0.2993 - val_loss: 1.6626 - val_categorical_accuracy: 0.3760
Epoch 3/60
100/100 [==============================] - 66s 658ms/step - loss: 1.4202 - categorical_accuracy: 0.4617 - val_loss: 1.1577 - val_categorical_accuracy: 0.5819
Epoch 4/60
100/100 [==============================] - 66s 657ms/step - loss: 1.1817 - categorical_accuracy: 0.5664 - val_loss: 1.0545 - val_categorical_accuracy: 0.6281
Epoch 5/60
100/100 [==============================] - 66s 662ms/step - loss: 0.9354 - categorical_accuracy: 0.6624 - val_loss: 0.7742 - val_categorical_accuracy: 0.7328
Epoch 6/60
100/100 [==============================] - 65s 653

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [0]:
model.save('/content/drive/My Drive/Kaggle/Speaker Recognition/model.h5')

In [0]:
from keras.applications.resnet50 import ResNet50
model2 = ResNet50(weights = "imagenet")



102858752/102853048 [==============================] - 1s 0us/step


In [0]:
model2.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
base_model = ResNet50(weights= "imagenet", include_top=False, input_shape= (224,224,3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 11s 0us/step


In [0]:
from keras import Model
from keras.layers import GlobalAveragePooling2D

In [0]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.27)(x)
x = Dense(64, activation = 'relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(10, activation= 'softmax')(x)
model_final = Model(inputs = base_model.input, outputs = predictions)

In [0]:
model_final.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
for layer in model_final.layers[:-4]:
  layer.trainable = False

In [0]:
model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=60,
        validation_data=valid_generator,
        validation_steps=70)

Epoch 1/60
100/100 [==============================] - 74s 744ms/step - loss: 0.1206 - categorical_accuracy: 0.9598 - val_loss: 0.3872 - val_categorical_accuracy: 0.8993
Epoch 2/60
100/100 [==============================] - 72s 718ms/step - loss: 0.1253 - categorical_accuracy: 0.9569 - val_loss: 0.4572 - val_categorical_accuracy: 0.8884
Epoch 3/60
100/100 [==============================] - 72s 717ms/step - loss: 0.1258 - categorical_accuracy: 0.9572 - val_loss: 0.4380 - val_categorical_accuracy: 0.8866
Epoch 4/60
100/100 [==============================] - 67s 673ms/step - loss: 0.1496 - categorical_accuracy: 0.9538 - val_loss: 0.4188 - val_categorical_accuracy: 0.9016
Epoch 5/60
100/100 [==============================] - 67s 668ms/step - loss: 0.1124 - categorical_accuracy: 0.9621 - val_loss: 0.3758 - val_categorical_accuracy: 0.9167
Epoch 6/60
100/100 [==============================] - 67s 671ms/step - loss: 0.1092 - categorical_accuracy: 0.9602 - val_loss: 0.4154 - val_categorical_acc

In [0]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

ValueError: ignored

In [0]:
model2.layers